In [2]:
import warnings
warnings.filterwarnings("ignore")
import csv
import pandas as pd
import datetime
import time
import numpy as np 
from time import time
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns 
from matplotlib import rcParams 
from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import pickle
import os
import networkx as nx
import pdb
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm
import xgboost as xgb
import random
import networkx as nx
from sklearn.model_selection import train_test_split



In [3]:
if os.path.isfile('train_data.csv'):
    train_graph=nx.read_edgelist('train_data.csv',delimiter=',',create_using=nx.DiGraph())
    print(nx.info(train_graph))


Name: 
Type: DiGraph
Number of nodes: 6552
Number of edges: 12109
Average in degree:   1.8481
Average out degree:   1.8481


# Features

In [ ]:
1.page rank

In [4]:
pr = nx.pagerank(train_graph, alpha=0.85)
pickle.dump(pr,open('page_rank.p','wb'))
mean_pr=float(sum(pr.values())) / len(pr)


In [ ]:
2.shortest path

In [5]:
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [ ]:
3.Weakly connected components

In [6]:
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

In [ ]:
4.Adar Index

In [7]:
def calc_adar_in(x,y):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [ ]:
5.Katz Centrality

In [8]:
katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
pickle.dump(katz,open('katz.p','wb'))
mean_katz= 0.012338187799665829
print('mean',float(sum(katz.values())) / len(katz))

mean 0.012338187799665829


In [ ]:
6.HITS

In [9]:
hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
pickle.dump(hits,open('hits.p','wb'))

In [10]:
filename = "train_final.csv"
#number of elements
n_train = sum(1 for line in open(filename)) 
s = 12110 # sample size
skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))

In [11]:
filename = "test_final.csv"
#number of elements
n_test = sum(1 for line in open(filename))
s = 3028 # sample size
skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))

In [12]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 24220
Number of rows we are going to elimiate in train data are 12110
Number of rows in the test data file: 6056
Number of rows we are going to elimiate in test data are 3028


In [13]:
df_final_train = pd.read_csv('train_final.csv', skiprows=skip_train, names=['drug', 'protein'])
df_final_train['indice'] = pd.read_csv('train_y.csv', skiprows=skip_train, names=['indice'])
print("Our train matrix size ",df_final_train.shape)


Our train matrix size  (12110, 3)


In [14]:
df_final_test = pd.read_csv('test_final.csv', skiprows=skip_test, names=['drug', 'protein'])
df_final_test['indice'] = pd.read_csv('test_y.csv', skiprows=skip_test, names=['indice'])
print("Our test matrix size ",df_final_test.shape)


Our test matrix size  (3029, 3)


In [32]:
df_final_train.head(7)

,drug,protein,indice
0,DB00184,P05181,1
1,DB06637,Q9NZV8,1
2,DB00201,Q01064,1
3,DB00141,Q9UJ70,1
4,DB00454,P10635,1
5,DB01291,P08588,1
6,DB00818,P47870,1


In [33]:
df_final_test.head(7)

,drug,protein,indice
0,DB00421,P06401,1
1,DB00175,P08684,1
2,DB01101,Q12882,1
3,DB06144,P50406,1
4,DB00590,P35348,1
5,DB02058,P21802,1
6,DB00731,P20815,1


In [38]:
file = open("page_rank.p","rb")
file1 = open("hits.p","rb")
file2 = open("katz.p","rb")

In [39]:
pr = pickle.load(file)
katz = pickle.load(file1)

In [48]:
#page rank for source and destination in Train and Test
#if anything not there in train graph then adding mean page rank 
df_final_train['page_rank_s'] = df_final_train.drug.apply(lambda x:pr.get(x,mean_pr))
df_final_train['page_rank_d'] = df_final_train.protein.apply(lambda x:pr.get(x,mean_pr))

df_final_test['page_rank_s'] = df_final_test.drug.apply(lambda x:pr.get(x,mean_pr))
df_final_test['page_rank_d'] = df_final_test.protein.apply(lambda x:pr.get(x,mean_pr))
    
#================================================================================
#mapping adar index on train
df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['drug'],row['protein']),axis=1)
#mapping adar index on test
df_final_test['adar_index'] = df_final_test.apply(lambda row: calc_adar_in(row['drug'],row['protein']),axis=1)

#================================================================================
#mapping same component of wcc or not on train
df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['drug'],row['protein']),axis=1)

##mapping same component of wcc or not on train
df_final_test['same_comp'] = df_final_test.apply(lambda row: belongs_to_same_wcc(row['drug'],row['protein']),axis=1)

#================================================================================
#mapping shortest path on train 
df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['drug'],row['protein']),axis=1)
#mapping shortest path on test
df_final_test['shortest_path'] = df_final_test.apply(lambda row: compute_shortest_path_length(row['drug'],row['protein']),axis=1)
#================================================================================

#Katz centrality score for source and destination in Train and test
#if anything not there in train graph then adding mean katz score
df_final_train['katz_s'] = df_final_train.drug.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.protein.apply(lambda x: katz.get(x,mean_katz))

df_final_test['katz_s'] = df_final_test.drug.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.protein.apply(lambda x: katz.get(x,mean_katz))
#================================================================================

#Hits algorithm score for source and destination in Train and test
#if anything not there in train graph then adding 0
df_final_train['hubs_s'] = df_final_train.drug.apply(lambda x: hits[0].get(x,0))
df_final_train['hubs_d'] = df_final_train.protein.apply(lambda x: hits[0].get(x,0))

df_final_test['hubs_s'] = df_final_test.drug.apply(lambda x: hits[0].get(x,0))
df_final_test['hubs_d'] = df_final_test.protein.apply(lambda x: hits[0].get(x,0))
#-------------------------------------------------------------------------------
df_final_train['authorities_s'] = df_final_train.drug.apply(lambda x: hits[1].get(x,0))
df_final_train['authorities_d'] = df_final_train.protein.apply(lambda x: hits[1].get(x,0))

df_final_test['authorities_s'] = df_final_test.drug.apply(lambda x: hits[1].get(x,0))
df_final_test['authorities_d'] = df_final_test.protein.apply(lambda x: hits[1].get(x,0))

#================================================================================

#saving the feature in 'Features.h5'
hdf = HDFStore('Features.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.put('test_df',df_final_test, format='table', data_columns=True)
hdf.close()
